# 필요 라이브러리 import

In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from wordcloud import wordcloud
%matplotlib inline
import pickle

# 딕셔너리 형 데이터 불러오기 & 처리

In [5]:
# load
with open('movie_dict.pickle', 'rb') as f:
    movie_dict = pickle.load(f)

In [9]:
# 딕셔너리를 데이터 프레임화 하기
df = pd.DataFrame(list(movie_dict.items()),columns = ['title','review'])  

In [10]:
# tf-idf 행렬만들기
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.review)

In [14]:
tfidf_matrix.shape # shape 확인

(7633, 55835)

# 코사인 유사도 만들기

In [17]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

array([1.        , 0.04777547, 0.03308296, ..., 0.02898799, 0.07744398,
       0.0289003 ])

# 영화이름을 인덱스로 만들기

In [18]:
indices = pd.Series(df.index, index = df.title) # 영화 제목으로 인덱스만들기
print(indices) 

title
카오산 탱고            0
욕창                1
바다로 가자            2
국도극장              3
레미: 집 없는 아이       4
               ... 
미트 페어런츠        7628
쥬브나일           7629
게임오버           7630
이유없는 반항        7631
천사의 시          7632
Length: 7633, dtype: int64


# 코사인 유사도 기반 영화 추천시스템 만들기

In [55]:
def movie_Recommendation(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True) # score 순으로 정렬

    sim_scores = sim_scores[1:11] # 가장 유사한 10개의 영화를 받아옴
    movie_indices = [i[0] for i in sim_scores] # 인덱스 받아오기
    
    result_df = df.iloc[movie_indices].copy()  #기존에 읽어들인 데이터에서 해당 인덱스의 값을 가져오기 스코어 열을 추가
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 가장 유사한 10개의 영화의 제목을 리턴
    return result_df


# 추천시스템 성능확인

In [66]:
movie_Recommendation('')

,title,review,score
4327,아무르,노인 영 추천 하다 없다 신기하다 일이 아무렇다 않다 대 터지다 나오다사랑 스며들다...,0.116228
3664,셔틀콕,남동생 가 셔틀콕 생각 하다 주다 가기 성가시다 줍다 존재 완벽하다 이 여운 있다 ...,0.112742
1012,봄이가도,의도 알다 자기 먹 여성 도움 필요하다 남성 클리셰 보고 싶다 다만 단편 트라우마 ...,0.112741
5578,걸어도 걸어도,살다 영 보다 복 걸다 좁다 않다 어도 늦다 되다 그렇다 노래 부르다 부산 놀다 가...,0.111792
57,애프터 웨딩 인 뉴욕,하나 특별하다 상상 하다 만큼 보여주다 이야기 이끌다 나가다 중심 되어다 인물 잘못...,0.106123
4189,송 포 유,뻔하다 내용 알다 감동 받다 울다 사실 저런 이야기 않다 이제 만나다 사람 추천 주...,0.104894
4782,씨민과 나데르의 별거,티끌 만들다 태산 가족 이름 벌이다 이기심 정직하다 하다 좋다 감독 사 족하 이면 ...,0.103452
304,니나 내나,살다보면 사정 저런 많다 연속 살아가다 사람 하다 가정 소소하다 특별하다 이야기 니...,0.103300
6196,아들,카메라 인물 마음 하나 되다 기적 같다 순간 도대체 누가 용서 구원 받다 섣부르다 ...,0.103123
925,친애하는 우리 아이,사정 각자 달라 여전하다 가족 있다 국내 개봉 하다 좋다 다시 보다 이유 기분 물어...,0.102859
